# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

The URL here has many query parameters

- `q` for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- `l` for a location 
- `start` for what result number to start on

In [1]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup

In [3]:
## YOUR CODE HERE

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- **Make sure these functions are robust and can handle cases where the data/field may not be available.**
    - Remember to check if a field is empty or `None` for attempting to call methods on it
    - Remember to use `try/except` if you anticipate errors
- **Test** the functions on the results above and simple examples

In [4]:
## YOUR CODE HERE

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [5]:
YOUR_CITY = ''

In [6]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.

results = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', YOUR_CITY]):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        # Append to the full set of results
        pass

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [7]:
## YOUR CODE HERE

Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries

In [8]:
## YOUR CODE HERE

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [9]:
## YOUR CODE HERE

### Save your results as a CSV

In [10]:
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/Karibenson/dsi/dsi-repo/DSI_SM_3/projects/project-04/assets/indeed-scraped-job-postings.csv')



## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [11]:
df.head()

,city,company,salary,summary,title,parsed_salary
0,Denver,Department Of The Interior,"$76,341 - $99,243 a year","Would you like to join the more than 10,000 sc...","Statistician, GS-1350-12 (DEU-PERM-DS)",87792.0
1,Denver,Department Of The Interior,"$71,012 - $99,243 a year",Investigate potential uses of geospatial data ...,Interdisciplinary Cartographer/Geographer - GS...,85127.5
2,Denver,Mental Health Center of Denver,"$70,000 - $80,000 a year",Advise the Data Developer with regard to creat...,Financial Data Scientist,75000.0
3,Denver,Denver Public Schools,"$62,712 - $75,255 a year",Portal managers on student outcome data report...,SENIOR RESEARCH ANALYST,68983.5
4,Denver,University of Colorado,"$25,000 - $29,000 a year",Experience entering and manipulating data in a...,Animal Care I,27000.0


#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't have to choice the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries, 

In [12]:
# get rid of NaNs
df['parsed_salary'].dropna(inplace=True)
df['parsed_salary'].unique()

array([  87792. ,   85127.5,   75000. ,   68983.5,   27000. ,   93645. ,
         66654. ,   77500. ,  130000. ,   85000. ,   55733.5,   43500. ,
         44250. ,  105000. ,   47500. ,  170000. ,  140000. ,  150000. ,
         35000. ,  160000. ,   41000. ,   65000. ,  120000. ,  135000. ,
         82500. ,  190000. ,  125000. ,   40000. ,   44293.5,   81078. ,
         71592. ,   87313.5,   59910. ,   80000. ,  110000. ,   90000. ,
         56000. ,   60000. ,  165000. ,  100000. ,   70000. ,  115285.5,
         10000. ,  115000. ,  145000. ,   74782.5,   79150. ,   42000. ,
         49832.5,   45162. ,   26442. ,   39390. ,   42723.5,   30264. ,
         23322. ,   60528. ,   50375. ,  107500. ,   32900. ,   95000. ,
        155000. ,   71000. ,  138300. ,  200000. ,  175000. ,   55000. ,
        185000. ,   50502.5,   50000. ,  180000. ,  137500. ,   82329. ,
         58000. ,  112500. ,   41600. ,   75500. ,   87368.5,   74317.5,
        132500. ,  122500. ,   85038.5,  225000. , 

In [13]:
df['city'].unique()

array(['Denver', 'Austin', 'Seattle', 'Philadelphia', 'Palo+Alto',
       'Pittsburgh', 'Miami', 'Chicago', 'Dallas', 'San+Diego', 'New+York',
       'Boston', 'Los+Angeles', 'Phoenix', 'Houston', 'Portland',
       'San+Francisco', 'Atlanta'], dtype=object)

In [14]:
# rename LA, SF, SD, NY
def renamed_cities(city):
    if city == 'Los+Angeles':
        return 'Los Angeles'
    elif city == 'San+Francisco':
        return 'San Francisco'
    elif city == 'San+Diego':
        return 'San Diego'
    elif city == 'Palo+Alto':
        return 'Palo Alto'
    elif city == 'New+York':
        return 'New York'
    else:
        return city
df['city'] = df['city'].apply(renamed_cities)   

In [15]:
df['city'].unique()

array(['Denver', 'Austin', 'Seattle', 'Philadelphia', 'Palo Alto',
       'Pittsburgh', 'Miami', 'Chicago', 'Dallas', 'San Diego', 'New York',
       'Boston', 'Los Angeles', 'Phoenix', 'Houston', 'Portland',
       'San Francisco', 'Atlanta'], dtype=object)

In [16]:
pivottable = pd.pivot_table(df, index=["city"], values=["parsed_salary"], 
                               aggfunc=(np.mean, np.median))
pivottable

parsed_salary           
                        mean     median
city                                   
Atlanta         85279.552632   80000.00
Austin          96571.428571   85000.00
Boston         116635.904255  117234.50
Chicago        118959.750000  120000.00
Dallas          97958.333333   90000.00
Denver          78196.441176   75000.00
Houston         64405.333333   67500.00
Los Angeles    112636.260870  101654.50
Miami           76795.833333   85000.00
New York       107889.147059   85019.25
Palo Alto      117267.333333  130000.00
Philadelphia   105545.454545  110000.00
Phoenix         55786.083333   44375.00
Pittsburgh      38658.666667   39390.00
Portland        78195.500000   78195.50
San Diego       94478.600000   81434.25
San Francisco  141321.521277  145000.00
Seattle        111225.210526  120000.00

In [17]:
median_salary = df['parsed_salary'].median()
median_salary

100000.0

In [18]:
mean_salary = df['parsed_salary'].mean()
mean_salary

107762.69581280788

#### Thought experiment: What is the baseline accuracy for this model?

Total mean salary and median salary are very similar.  To keep things simple, we will be using the median salary of $100,000.  This means our baseline accuracy would be 50% - 50% of the jobs fall below and above the median salary.

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [19]:
import statsmodels.formula.api as smf

In [20]:
# create target vector column.
# add binary values for salaries. 1 if median salary is above $100,000, 0 if it is below.
def high_salary(salary):
    if salary > df['parsed_salary'].median():
        return 1
    else:
        return 0
    
df['binary_salary']=df['parsed_salary'].apply(high_salary)



In [21]:
model = smf.ols('binary_salary ~ city', data=df).fit().summary()
model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          binary_salary   R-squared:                       0.168
Model:                            OLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     4.707
Date:                Sun, 15 Jan 2017   Prob (F-statistic):           4.00e-09
Time:                        23:26:05   Log-Likelihood:                -261.31
No. Observations:                 413   AIC:                             558.6
Df Residuals:                     395   BIC:                             631.0
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
Intercept                 0.2105      0.107      1.970      0.050         0.000     0.421
city[T.Austin]            0.2180      0.206      1.059      0.290        -0.187     0.623
city[T.Boston]            0.4703      0.127      3.714      0.000         0.221     0.719
city[T.Chicago]           0.3173      0.132      2.402      0.017         0.058     0.577
city[T.Dallas]            0.1228      0.172      0.715      0.475        -0.215     0.460
city[T.Denver]           -0.0341      0.156     -0.219      0.827        -0.340     0.272
city[T.Houston]          -0.2105      0.188     -1.117      0.265        -0.581     0.160
city[T.Los Angeles]       0.2895      0.143      2.024      0.044         0.008     0.571
city[T.Miami]            -0.0439      0.218     -0.201      0.841        -0.473     0.385
city[T.New York]          0.2069      0.116      1.779      0.076        -0.022     0.436
city[T.Palo Alto]         0.4932      0.139      3.536      0.000         0.219     0.767
city[T.Philadelphia]      0.4258      0.176      2.413      0.016         0.079     0.773
city[T.Phoenix]          -0.2105      0.218     -0.965      0.335        -0.639     0.218
city[T.Pittsburgh]       -0.2105      0.188     -1.117      0.265        -0.581     0.160
city[T.Portland]         -0.2105      0.478     -0.440      0.660        -1.150     0.729
city[T.San Diego]         0.1895      0.182      1.041      0.298        -0.168     0.547
city[T.San Francisco]     0.5446      0.126      4.326      0.000         0.297     0.792
city[T.Seattle]           0.2895      0.149      1.940      0.053        -0.004     0.583
==============================================================================
Omnibus:                        0.201   Durbin-Watson:                   1.822
Prob(Omnibus):                  0.905   Jarque-Bera (JB):               35.898
Skew:                          -0.053   Prob(JB):                     1.60e-08
Kurtosis:                       1.560   Cond. No.                         23.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The coefficients listed for each city indicate how the average salary for that city relates to the median salary. A positive value coeffiecient indicated the city's average salary is higher than the median, and lower would be less.  The absolute value of the coefficients indicates the magnitude of how much greater than or less than that city's average salary falls from the median.

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [22]:
# find all titles with Senior or SR
def lower(x):
    return x.lower()
df['title']=df['title'].apply(lower)
df['contains_senior'] = df['title'].str.contains('senior','sr')
df.head()

,city,company,salary,summary,title,parsed_salary,binary_salary,contains_senior
0,Denver,Department Of The Interior,"$76,341 - $99,243 a year","Would you like to join the more than 10,000 sc...","statistician, gs-1350-12 (deu-perm-ds)",87792.0,0,False
1,Denver,Department Of The Interior,"$71,012 - $99,243 a year",Investigate potential uses of geospatial data ...,interdisciplinary cartographer/geographer - gs...,85127.5,0,False
2,Denver,Mental Health Center of Denver,"$70,000 - $80,000 a year",Advise the Data Developer with regard to creat...,financial data scientist,75000.0,0,False
3,Denver,Denver Public Schools,"$62,712 - $75,255 a year",Portal managers on student outcome data report...,senior research analyst,68983.5,0,True
4,Denver,University of Colorado,"$25,000 - $29,000 a year",Experience entering and manipulating data in a...,animal care i,27000.0,0,False


In [23]:
# find all managemenet-level titles 
def lower(x):
    return x.lower()
df['title']=df['title'].apply(lower)
df['contains_manager'] = df['title'].str.contains('manager','mgr')
df.head()

,city,company,salary,summary,title,parsed_salary,binary_salary,contains_senior,contains_manager
0,Denver,Department Of The Interior,"$76,341 - $99,243 a year","Would you like to join the more than 10,000 sc...","statistician, gs-1350-12 (deu-perm-ds)",87792.0,0,False,False
1,Denver,Department Of The Interior,"$71,012 - $99,243 a year",Investigate potential uses of geospatial data ...,interdisciplinary cartographer/geographer - gs...,85127.5,0,False,False
2,Denver,Mental Health Center of Denver,"$70,000 - $80,000 a year",Advise the Data Developer with regard to creat...,financial data scientist,75000.0,0,False,False
3,Denver,Denver Public Schools,"$62,712 - $75,255 a year",Portal managers on student outcome data report...,senior research analyst,68983.5,0,True,False
4,Denver,University of Colorado,"$25,000 - $29,000 a year",Experience entering and manipulating data in a...,animal care i,27000.0,0,False,False


In [24]:
model2 = smf.ols('binary_salary ~ contains_senior', data=df).fit().summary()
model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          binary_salary   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     7.431
Date:                Sun, 15 Jan 2017   Prob (F-statistic):            0.00669
Time:                        23:26:05   Log-Likelihood:                -295.70
No. Observations:                 413   AIC:                             595.4
Df Residuals:                     411   BIC:                             603.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------------
Intercept                   0.4517      0.026     17.075      0.000         0.400     0.504
contains_senior[T.True]     0.1876      0.069      2.726      0.007         0.052     0.323
==============================================================================
Omnibus:                        0.572   Durbin-Watson:                   1.548
Prob(Omnibus):                  0.751   Jarque-Bera (JB):               64.164
Skew:                           0.090   Prob(JB):                     1.17e-14
Kurtosis:                       1.077   Cond. No.                         2.89
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
model3 = smf.ols('binary_salary ~ contains_manager', data=df).fit().summary()
model3

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          binary_salary   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.6704
Date:                Sun, 15 Jan 2017   Prob (F-statistic):              0.413
Time:                        23:26:06   Log-Likelihood:                -299.07
No. Observations:                 413   AIC:                             602.1
Df Residuals:                     411   BIC:                             610.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------
Intercept                    0.4845      0.025     19.074      0.000         0.435     0.534
contains_manager[T.True]    -0.0845      0.103     -0.819      0.413        -0.288     0.118
==============================================================================
Omnibus:                        0.474   Durbin-Watson:                   1.535
Prob(Omnibus):                  0.789   Jarque-Bera (JB):               68.397
Skew:                           0.082   Prob(JB):                     1.41e-15
Kurtosis:                       1.013   Cond. No.                         4.21
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Titles containing Senior have more influence on whether or not salary is high than titles containing manager.

In [26]:
model4 = smf.ols('binary_salary ~ city + contains_senior + contains_manager', data=df).fit().summary()
model4

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          binary_salary   R-squared:                       0.180
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     4.552
Date:                Sun, 15 Jan 2017   Prob (F-statistic):           1.99e-09
Time:                        23:26:06   Log-Likelihood:                -258.33
No. Observations:                 413   AIC:                             556.7
Df Residuals:                     393   BIC:                             637.1
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------
Intercept                    0.2121      0.106      1.992      0.047         0.003     0.421
city[T.Austin]               0.1942      0.205      0.946      0.345        -0.209     0.598
city[T.Boston]               0.4408      0.127      3.479      0.001         0.192     0.690
city[T.Chicago]              0.3009      0.132      2.284      0.023         0.042     0.560
city[T.Dallas]               0.0978      0.171      0.570      0.569        -0.239     0.435
city[T.Denver]              -0.0394      0.155     -0.254      0.800        -0.345     0.266
city[T.Houston]             -0.2434      0.188     -1.293      0.197        -0.614     0.127
city[T.Los Angeles]          0.2761      0.142      1.938      0.053        -0.004     0.556
city[T.Miami]               -0.0974      0.218     -0.446      0.656        -0.527     0.332
city[T.New York]             0.1880      0.116      1.620      0.106        -0.040     0.416
city[T.Palo Alto]            0.4742      0.139      3.409      0.001         0.201     0.748
city[T.Philadelphia]         0.3959      0.176      2.248      0.025         0.050     0.742
city[T.Phoenix]             -0.2121      0.217     -0.977      0.329        -0.639     0.215
city[T.Pittsburgh]          -0.2295      0.188     -1.222      0.223        -0.599     0.140
city[T.Portland]            -0.3681      0.480     -0.766      0.444        -1.312     0.576
city[T.San Diego]            0.1255      0.183      0.685      0.494        -0.234     0.485
city[T.San Francisco]        0.5060      0.126      4.005      0.000         0.258     0.754
city[T.Seattle]              0.2753      0.149      1.851      0.065        -0.017     0.568
contains_senior[T.True]      0.1559      0.067      2.345      0.020         0.025     0.287
contains_manager[T.True]    -0.0305      0.097     -0.314      0.754        -0.222     0.161
==============================================================================
Omnibus:                     3358.901   Durbin-Watson:                   1.826
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               33.465
Skew:                          -0.043   Prob(JB):                     5.41e-08
Kurtosis:                       1.608   Cond. No.                         24.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
model5 = smf.ols('binary_salary ~ city + contains_senior', data=df).fit().summary()
model5

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          binary_salary   R-squared:                       0.180
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     4.811
Date:                Sun, 15 Jan 2017   Prob (F-statistic):           9.18e-10
Time:                        23:26:06   Log-Likelihood:                -258.38
No. Observations:                 413   AIC:                             554.8
Df Residuals:                     394   BIC:                             631.2
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------------
Intercept                   0.2105      0.106      1.981      0.048         0.002     0.419
city[T.Austin]              0.1956      0.205      0.954      0.341        -0.207     0.599
city[T.Boston]              0.4402      0.127      3.479      0.001         0.191     0.689
city[T.Chicago]             0.2998      0.132      2.279      0.023         0.041     0.558
city[T.Dallas]              0.0966      0.171      0.564      0.573        -0.240     0.433
city[T.Denver]             -0.0433      0.155     -0.280      0.780        -0.347     0.261
city[T.Houston]            -0.2455      0.188     -1.306      0.192        -0.615     0.124
city[T.Los Angeles]         0.2764      0.142      1.942      0.053        -0.003     0.556
city[T.Miami]              -0.0963      0.218     -0.442      0.659        -0.525     0.332
city[T.New York]            0.1871      0.116      1.614      0.107        -0.041     0.415
city[T.Palo Alto]           0.4757      0.139      3.425      0.001         0.203     0.749
city[T.Philadelphia]        0.3972      0.176      2.259      0.024         0.051     0.743
city[T.Phoenix]            -0.2105      0.217     -0.971      0.332        -0.637     0.216
city[T.Pittsburgh]         -0.2280      0.188     -1.216      0.225        -0.597     0.141
city[T.Portland]           -0.3679      0.480     -0.767      0.444        -1.311     0.575
city[T.San Diego]           0.1265      0.183      0.692      0.489        -0.233     0.486
city[T.San Francisco]       0.5060      0.126      4.010      0.000         0.258     0.754
city[T.Seattle]             0.2737      0.149      1.843      0.066        -0.018     0.566
contains_senior[T.True]     0.1574      0.066      2.374      0.018         0.027     0.288
==============================================================================
Omnibus:                     3333.827   Durbin-Watson:                   1.827
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               33.441
Skew:                          -0.040   Prob(JB):                     5.47e-08
Kurtosis:                       1.608   Cond. No.                         24.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Looking at information from the job title alone does not give us a better model for predicted salary.  Higher salaries seem to be mostly correlated with the city of the job rather than the job title.  We can get a slightly better model when looking at the city and whether or not 'Senior' is in the job title.

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [28]:
df=pd.concat([df, pd.get_dummies(df['city'], prefix='city_', drop_first=True)], axis=1)
df.head(2)

,city,company,salary,summary,title,parsed_salary,binary_salary,contains_senior,contains_manager,city__Austin,...,city__Miami,city__New York,city__Palo Alto,city__Philadelphia,city__Phoenix,city__Pittsburgh,city__Portland,city__San Diego,city__San Francisco,city__Seattle
0,Denver,Department Of The Interior,"$76,341 - $99,243 a year","Would you like to join the more than 10,000 sc...","statistician, gs-1350-12 (deu-perm-ds)",87792.0,0,False,False,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Denver,Department Of The Interior,"$71,012 - $99,243 a year",Investigate potential uses of geospatial data ...,interdisciplinary cartographer/geographer - gs...,85127.5,0,False,False,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
y = df['binary_salary']
X = df.loc[:,'city__Austin':'city__Seattle']

In [35]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


def least_squares_regressor_standard_workflow(X, y, random_state=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state)
    model_sklearn = LinearRegression()
    model_sklearn.fit(X_train, y_train)
    train_score = model_sklearn.score(X_train, y_train)
    test_score = model_sklearn.score(X_test, y_test)
    return {
        'model' : model,
        'train_score' : train_score,
        'test_score' : test_score}

In [36]:
from sklearn.preprocessing import StandardScaler

def scaler_standard_workflow(X, y, random_state=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return {
        'scaler' : scaler,
        'X_test' : X_test,
        'X_train' : X_train,
        'y_test' : y_test,
        'y_train' : y_train}

In [37]:
least_squares_regressor_standard_workflow(X, y)

{'model': <class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:          binary_salary   R-squared:                       0.168
 Model:                            OLS   Adj. R-squared:                  0.133
 Method:                 Least Squares   F-statistic:                     4.707
 Date:                Sun, 15 Jan 2017   Prob (F-statistic):           4.00e-09
 Time:                        23:26:05   Log-Likelihood:                -261.31
 No. Observations:                 413   AIC:                             558.6
 Df Residuals:                     395   BIC:                             631.0
 Df Model:                          17                                         
 Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
 -------------------------------------------------

In [38]:
scaler_standard_workflow(X, y)

{'X_test': array([[-0.11451967, -0.37446148, -0.32182515, ..., -0.14071951,
         -0.39113307,  4.14445058],
        [-0.11451967, -0.37446148, -0.32182515, ..., -0.14071951,
         -0.39113307, -0.24128651],
        [-0.11451967, -0.37446148, -0.32182515, ...,  7.1063352 ,
         -0.39113307, -0.24128651],
        ..., 
        [ 8.7321246 , -0.37446148, -0.32182515, ..., -0.14071951,
         -0.39113307, -0.24128651],
        [-0.11451967, -0.37446148, -0.32182515, ..., -0.14071951,
         -0.39113307, -0.24128651],
        [-0.11451967, -0.37446148, -0.32182515, ..., -0.14071951,
         -0.39113307, -0.24128651]]),
 'X_train': array([[-0.11451967,  2.67050163, -0.32182515, ..., -0.14071951,
         -0.39113307, -0.24128651],
        [-0.11451967, -0.37446148, -0.32182515, ..., -0.14071951,
         -0.39113307, -0.24128651],
        [-0.11451967, -0.37446148, -0.32182515, ..., -0.14071951,
         -0.39113307, -0.24128651],
        ..., 
        [-0.11451967, -0.374461

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [ ]:
## YOUR CODE HERE

# 4.4.1

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score

In [40]:
knn_params = {
    'n_neighbors':range(1,101,2),
    'weights':['uniform','distance']
}

knn_gs = GridSearchCV(KNeighborsClassifier(), knn_params, cv=10, verbose=1)
knn_gs.fit(X, y)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   30.1s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99], 'weights': ['uniform', 'distance']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [41]:
best_knn = knn_gs.best_estimator_

In [42]:
knn_gs.best_params_

{'n_neighbors': 3, 'weights': 'uniform'}

In [43]:
knn_gs.best_score_

0.63196125907990319

In [44]:
1 - np.mean(y)

0.5205811138014528

#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [45]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
Xn = ss.fit_transform(X)

In [46]:
lr = LogisticRegression()

lr_params = {
    'penalty':['l1','l2'],
    'solver':['liblinear'],
    'C':np.linspace(0.0001, 10, 200)
}

lr_gs = GridSearchCV(lr, lr_params, cv=10, verbose=1)
lr_gs.fit(Xn, y)

Fitting 10 folds for each of 400 candidates, totalling 4000 fits


[Parallel(n_jobs=1)]: Done 4000 out of 4000 | elapsed:   52.0s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': array([  1.00000e-04,   5.03508e-02, ...,   9.94975e+00,   1.00000e+01]), 'solver': ['liblinear']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [47]:
best_lr = lr_gs.best_estimator_
print lr_gs.best_params_
print lr_gs.best_score_

{'penalty': 'l2', 'C': 0.0001, 'solver': 'liblinear'}
0.624697336562


In [50]:
y_preds = pd.DataFrame({
        'y_true':y,
        'y_hat':best_lr.predict(Xn),
        'positive_pp':best_lr.predict_proba(Xn)[:,1]
    })
y_preds.head()

,positive_pp,y_hat,y_true
0,0.49641,0,0
1,0.49641,0,0
2,0.49641,0,0
3,0.49641,0,0
4,0.49641,0,0


In [51]:
coefs_vars = pd.DataFrame({
        'coef':best_lr.coef_[0],
        'variable':X.columns,
        'abscoef':np.abs(best_lr.coef_[0])
    })
coefs_vars.sort_values('abscoef', ascending=False, inplace=True)
coefs_vars[coefs_vars.coef != 0]

,abscoef,coef,variable
15,0.004131,0.004131,city__San Francisco
1,0.002949,0.002949,city__Boston
5,0.002923,-0.002923,city__Houston
12,0.002923,-0.002923,city__Pittsburgh
4,0.002563,-0.002563,city__Denver
9,0.002425,0.002425,city__Palo Alto
11,0.002378,-0.002378,city__Phoenix
7,0.001551,-0.001551,city__Miami
8,0.001451,-0.001451,city__New York
10,0.001062,0.001062,city__Philadelphia


#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

In [58]:
def lower(x):
    return x.lower()
df['summary']=df['summary'].apply(lower)
df['high_education'] = df['summary'].str.contains('phd', 'masters')
df.head()

,city,company,salary,summary,title,parsed_salary,binary_salary,contains_senior,contains_manager,city__Austin,...,city__Palo Alto,city__Philadelphia,city__Phoenix,city__Pittsburgh,city__Portland,city__San Diego,city__San Francisco,city__Seattle,contains_phd,high_education
0,Denver,Department Of The Interior,"$76,341 - $99,243 a year","would you like to join the more than 10,000 sc...","statistician, gs-1350-12 (deu-perm-ds)",87792.0,0,False,False,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False
1,Denver,Department Of The Interior,"$71,012 - $99,243 a year",investigate potential uses of geospatial data ...,interdisciplinary cartographer/geographer - gs...,85127.5,0,False,False,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False
2,Denver,Mental Health Center of Denver,"$70,000 - $80,000 a year",advise the data developer with regard to creat...,financial data scientist,75000.0,0,False,False,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False
3,Denver,Denver Public Schools,"$62,712 - $75,255 a year",portal managers on student outcome data report...,senior research analyst,68983.5,0,True,False,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False
4,Denver,University of Colorado,"$25,000 - $29,000 a year",experience entering and manipulating data in a...,animal care i,27000.0,0,False,False,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False


In [57]:
model6 = smf.ols('binary_salary ~ high_education', data=df).fit().summary()
model6

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          binary_salary   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.2936
Date:                Sun, 15 Jan 2017   Prob (F-statistic):              0.588
Time:                        23:50:09   Log-Likelihood:                -299.25
No. Observations:                 413   AIC:                             602.5
Df Residuals:                     411   BIC:                             610.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                  0.4779      0.025     19.284      0.000         0.429     0.527
high_education[T.True]     0.1221      0.225      0.542      0.588        -0.321     0.565
==============================================================================
Omnibus:                        0.485   Durbin-Watson:                   1.535
Prob(Omnibus):                  0.785   Jarque-Bera (JB):               68.644
Skew:                           0.083   Prob(JB):                     1.24e-15
Kurtosis:                       1.010   Cond. No.                         9.15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
def lower(x):
    return x.lower()
df['summary']=df['summary'].apply(lower)
df['machine_learning'] = df['summary'].str.contains('machine learning')
df.head()

,city,company,salary,summary,title,parsed_salary,binary_salary,contains_senior,contains_manager,city__Austin,...,city__Philadelphia,city__Phoenix,city__Pittsburgh,city__Portland,city__San Diego,city__San Francisco,city__Seattle,contains_phd,high_education,machine_learning
0,Denver,Department Of The Interior,"$76,341 - $99,243 a year","would you like to join the more than 10,000 sc...","statistician, gs-1350-12 (deu-perm-ds)",87792.0,0,False,False,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
1,Denver,Department Of The Interior,"$71,012 - $99,243 a year",investigate potential uses of geospatial data ...,interdisciplinary cartographer/geographer - gs...,85127.5,0,False,False,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
2,Denver,Mental Health Center of Denver,"$70,000 - $80,000 a year",advise the data developer with regard to creat...,financial data scientist,75000.0,0,False,False,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
3,Denver,Denver Public Schools,"$62,712 - $75,255 a year",portal managers on student outcome data report...,senior research analyst,68983.5,0,True,False,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
4,Denver,University of Colorado,"$25,000 - $29,000 a year",experience entering and manipulating data in a...,animal care i,27000.0,0,False,False,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False


In [60]:
model7 = smf.ols('binary_salary ~ machine_learning', data=df).fit().summary()
model7

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          binary_salary   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     8.129
Date:                Sun, 15 Jan 2017   Prob (F-statistic):            0.00458
Time:                        23:52:07   Log-Likelihood:                -295.36
No. Observations:                 413   AIC:                             594.7
Df Residuals:                     411   BIC:                             602.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------
Intercept                    0.4629      0.025     18.458      0.000         0.414     0.512
machine_learning[T.True]     0.3098      0.109      2.851      0.005         0.096     0.523
==============================================================================
Omnibus:                        0.738   Durbin-Watson:                   1.538
Prob(Omnibus):                  0.691   Jarque-Bera (JB):               64.645
Skew:                           0.102   Prob(JB):                     9.17e-15
Kurtosis:                       1.073   Cond. No.                         4.47
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [ ]:
## YOUR CODE HERE

In [ ]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [ ]:
## YOUR CODE HERE